In [2]:
from kendall_tau import compute_kendall_tau

taus = compute_kendall_tau("data/processed/train_processed.csv", "data/processed/train_reordered_pairs.csv")

print("Average Kendall tau:", sum(taus)/len(taus))

Average Kendall tau: 0.35985454545453716


In [2]:
from story_position_acc import count_sentence_mismatches, get_sentence_2_3_mismatches

mismatches = count_sentence_mismatches("data/processed/train_reordered_pairs.csv")
print("Total sentence mismatches:", mismatches)
print("Average sentence accuracy per story:", [(1375-m) / 1375 for m in mismatches])

examples = get_sentence_2_3_mismatches("data/processed/train_reordered_pairs.csv", k=10)
for e in examples:
    print("ORIG: ", e["orig"])
    print("REORD:", e["reord"])
    print()


Total sentence mismatches: [188, 272, 239, 206, 120]
Average sentence accuracy per story: [0.8632727272727273, 0.8021818181818182, 0.8261818181818181, 0.8501818181818181, 0.9127272727272727]
ORIG:  ['I asked for a drink afterwards, but he told me no', 'I was forced to do several push-ups']
REORD: ['He told me to stop as soon as I fell on the floor', 'I was forced to do several push-ups']

ORIG:  ['She sighed in frustration that she had ruined it', 'Picking it up, she put it back in the car']
REORD: ['The soda hit the ground hard', 'Picking it up, she put it back in the car']

ORIG:  ['She had been playing with his puppy before he got angry', 'She ran for blocks without looking back']
REORD: ['Lucy was running from an angry old man', 'She ran for blocks without looking back']

ORIG:  ['His doctor asked Max if he slept through the night regularly', "Max couldn't remember the last time he had slept through the night"]
REORD: ['Max noticed he was always tired so he went to the doctor for h